# [**Предсказание оттока пользователей**](https://www.kaggle.com/c/advanced-dls-spring-2021/)

In [ ]:
!gdown 1ERwQ5odiK1Zvi1LtjpkzCMUswYsAX8_K  # train.csv
!gdown 1fGw_-RFwvn_LEdt91Jq-7A-wzG6mmH8r  # test.csv
!gdown 199Mt4OYZNaelT83U-HGDsEYs2YcUGQ6y  # submission.csv

In [39]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier
from catboost import Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

# **Предварительная обработка данных**

Проверим количество NaN значений. Устраним, в случае необходимости

In [ ]:
df = pd.read_csv('./train.csv')
df.isna().sum()

Теперь выведем все значения для категориальных признаков, а также их количество

In [ ]:
columns = df.columns.tolist()[3:-1]

for column in columns:
  print(df[column].value_counts())
  print()

NaN отсутсвует

# **Графики**

Построим график взаимосвязи длительности использования услуг с ежемесячными тратами. Зелеными обозначим клиентов, которые остались. Красным - ушедших.

In [ ]:
retained_x = df[df['Churn'] == 0]['ClientPeriod']
retained_y = df[df['Churn'] == 0]['MonthlySpending']

churned_x = df[df['Churn'] == 1]['ClientPeriod']
churned_y = df[df['Churn'] == 1]['MonthlySpending']

plt.figure(figsize=(10,10))

plt.scatter(retained_x, retained_y, c='green', label='Оставшиеся клиенты')

plt.scatter(churned_x, churned_y, c='red', label='Ушедшие клиенты')


plt.legend()
plt.xlabel('Длительность сотрудничества')
plt.ylabel('Месячные траты')

plt.title('Длительность сотрудничества / Месячные траты')


plt.show()

Видим, что подавляющее число ушедших клиентов:


1.   Пользовались сервисом непродолжительное время
2.   Платили большую сумму


Построим боксплот для ClientPeriod

In [ ]:
plt.boxplot([df.loc[df['Churn'] == 0, 'ClientPeriod'], df.loc[df['Churn'] == 1, 'ClientPeriod']])


plt.xticks([1, 2], ['Оставшиеся клиениы', 'Ушедшие клиенты'])
plt.xlabel('Группы клиентов')
plt.ylabel('Срок сотрудничества')

plt.show()

Этот график более наглядно демонстрирует, что новые клиенты уходят чаще.

Построим боксплот для ежемесячных трат

In [ ]:
plt.boxplot([df.loc[df['Churn'] == 0, 'MonthlySpending'], df.loc[df['Churn'] == 1, 'MonthlySpending']])
plt.xticks([1,2], ['Оставшиеся клиениы', 'Ушедшие клиенты'])
plt.xlabel('Группы клиентов')
plt.ylabel('Ежемесячные траты')
plt.show()

Присутсвует небольшая разница в ежемесячной оплате.

Перейдем к категориальным признакам

In [94]:
columns_list = df.iloc[:, 3:-2].columns.tolist()

bins_name = []
bins_value = []

for column in columns_list:
  for category in df[column].unique().tolist():
    bins_name.append(str(column) + ' ' + str(category))
  for value in df[column].value_counts().tolist():
    bins_value.append(value)

In [ ]:

plt.figure(figsize=(10, 10))
plt.barh(bins_name, bins_value, height = 0.5)

plt.title('Категориальные признаки')
plt.xlabel('Количество')
plt.ylabel('Категории')

plt.show()

А теперь посмотрим на распределение таргета

In [ ]:
churned_count = df['Churn'].tolist().count(1)
retained_count = df['Churn'].tolist().count(0)

plt.pie([retained_count, churned_count], labels=['Клиенты остались', 'Клиенты покинули'])

plt.title('Распределение целевой переменной')

plt.show()

# **Логистическая регрессия**

Отнормируем признаки


In [66]:
df['TotalSpent'] = pd.to_numeric(df['TotalSpent'], errors='coerce')
numeric_columns = df.iloc[:, :3]
scaler = StandardScaler()

numeric_columns = scaler.fit_transform(numeric_columns)

df.iloc[:, :3] = numeric_columns


Разберемся с категориальными

In [67]:
categorical_columns = df.iloc[:, 3:-1]

categorical_columns = categorical_columns.drop(columns=df.columns[4])


encoded = pd.get_dummies(categorical_columns, prefix=categorical_columns.columns)


df = pd.concat([df.iloc[:, :3], encoded, df.iloc[:, 4], df.iloc[:, -1]], axis=1)



Запускаем кросс-валидацию.

p.s.
Уже на моменте обучения выяснилось, что TotalSpent после перевода во float будет иметь NaN. Так как число не существено (9), было решено просто удалить строки

In [68]:
df = df.dropna()
params = {
    'C': [100, 10, 1, 0.1, 0.01, 0.001]
}

classifier  = LogisticRegression(max_iter=1000)

grid_search = GridSearchCV(classifier, params, scoring='roc_auc', cv=5, refit=True)

train_X = df.iloc[:, :-1]
train_y = df.iloc[:, -1]

grid_search.fit(train_X, train_y)



GridSearchCV(cv=5, estimator=LogisticRegression(max_iter=1000),
             param_grid={'C': [100, 10, 1, 0.1, 0.01, 0.001]},
             scoring='roc_auc')

In [ ]:
print("Лучшие гиперпараметры:", grid_search.best_params_)
print("ROC AUC наилучшей модели:", grid_search.best_score_)

Лучшие гиперпараметры: {'C': 100}

ROC AUC наилучшей модели: 0.8448143844842171




# **Градиентный бустинг**

In [46]:
X = df.drop('Churn', axis=1)
y = df['Churn']

categorical_features_indices = list(range(3, len(df.columns) - 1))


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


train_pool = Pool(data=X_train, label=y_train, cat_features=categorical_features_indices)
test_pool = Pool(data=X_test, label=y_test, cat_features=categorical_features_indices)

kitty = CatBoostClassifier()
kitty.fit(train_pool, eval_set=test_pool, verbose=False)



In [42]:
boost_preds_test = kitty.predict_proba(X_test)[:,1]
CBST_simple_roc_auc = roc_auc_score(y_test, boost_preds_test)
CBST_simple_roc_auc


0.828497150931373

In [43]:
print("CatBoost со стандартными настройками:")
print("Значение AUC-ROC:", CBST_simple_roc_auc)

CatBoost со стандартными настройками:
Значение AUC-ROC: 0.828497150931373


Пробуем тестировать значения

In [ ]:
cat = CatBoostClassifier(iterations = 100, learning_rate=0.3)
cat.fit(train_pool, eval_set=test_pool, verbose=False)

boost_preds_test = cat.predict_proba(X_test)[:,1]
CBST_simple_roc_auc = roc_auc_score(y_test, boost_preds_test)
CBST_simple_roc_auc


Лучшая модель со стандартными настройками:

AUC-ROC: 0.828497150931373

# **Предсказание**

In [71]:
cat

X_test = pd.read_csv('./test.csv')
submission = pd.read_csv('./submission.csv')

submission['Churn'] = cat.predict(X_test)
submission.to_csv('./my_submission.csv')